In [53]:
import pandas as pd
from z.score import Metrics
import numpy as np

In [54]:
CSV = 'test.csv'

In [55]:
df = pd.read_csv(CSV, index_col=0)
# 将 object 类的 date 转化为 datetime
df['date'] = pd.to_datetime(df['date'])
print(df.dtypes)

date     datetime64[ns]
value           float64
dtype: object


In [56]:
"""计算逐天差"""
df['delta'] = df.value - df.value.shift(periods=1, fill_value=1.)

In [57]:
df['week'] = df['date'].dt.dayofweek + 1
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

week_coeff = []
month_coeff = []

print('星期一到星期五的累计涨幅')
for week, group in df.groupby('week'):
    print(week, group['delta'].sum())
    week_coeff.append(group['delta'].sum())
print('月的的累计涨幅')
for year, group in df.groupby('month'):
    print(year, group['delta'].sum())
    month_coeff.append(group['delta'].sum())
print('年的累计涨幅')
for year, group in df.groupby('year'):
    print(year, group['delta'].sum())
    month_coeff.append(group['delta'].sum())
# norm
norm_month_coeff = Metrics.norm(np.array(month_coeff)).tolist()
norm_week_coeff = Metrics.norm(np.array(week_coeff)).tolist()

星期一到星期五的累计涨幅
1 0.48579999999999934
2 0.03560000000000185
3 0.23089999999999833
4 -0.01579999999999937
5 -0.17120000000000024
月的的累计涨幅
1 -0.07930000000000015
2 -0.050000000000000044
3 -0.3535999999999999
4 0.16780000000000017
5 0.21860000000000013
6 0.6396999999999999
7 0.1427999999999996
8 -0.2991999999999997
9 -0.19240000000000035
10 0.16220000000000012
11 0.08060000000000023
12 0.12809999999999988
年的累计涨幅
2019 0.02760000000000007
2020 0.8808
2021 0.3084999999999998
2022 -0.6516


In [58]:
df

,date,value,delta,week,year,month
0,2019-11-20,1.0000,0.0000,3,2019,11
1,2019-11-22,1.0000,0.0000,5,2019,11
2,2019-11-29,0.9997,-0.0003,5,2019,11
3,2019-12-06,1.0043,0.0046,5,2019,12
4,2019-12-13,1.0117,0.0074,5,2019,12
...,...,...,...,...,...,...
664,2022-09-16,1.5899,-0.0403,5,2022,9
665,2022-09-19,1.5796,-0.0103,1,2022,9
666,2022-09-20,1.5964,0.0168,2,2022,9
667,2022-09-21,1.5741,-0.0223,3,2022,9


In [59]:
"""中位线计算"""
df_3_month = df[-90:]
df_year = df[-365:]
df_all = df

In [60]:
quantile_3_month = df_3_month.value.quantile([0.25, 0.5, 0.75])
quantile_year = df_year.value.quantile([0.25, 0.5, 0.75])
quantile_all = df_all.value.quantile([0.25, 0.5, 0.75])

In [61]:
quantile = [a*0.3 + b*0.5 + c*0.2 for a,b,c in zip(quantile_3_month, quantile_year, quantile_all)]

In [62]:
quantile

[1.7332524999999999, 1.87586, 2.0768775]

In [63]:
from datetime import datetime


def is_buy(value):
    """buy: True, sell: False"""
    if value < quantile[0]:
        return True
    elif value < quantile[1]:
        return False
    elif value < quantile[2]:
        return True
    else:
        return False


print(is_buy(1.5653))
def get_cycle(date: datetime):
    return 0.3 * norm_month_coeff[date.month - 1] + 0.7 * norm_week_coeff[date.weekday()]
print(get_cycle(datetime.strptime('2022-9-21', '%Y-%m-%d')))

True
0.5183152460878496


In [64]:
today = datetime.strptime('2022-9-21', '%Y-%m-%d')
cycle = get_cycle(today)
# cycle 是定投系数
# 定投考虑三者系数比

900.3628206722615

In [ ]:
"""卖出"""
# 卖出考虑本金，本星期浮动，下行趋势中的小涨是下船的好时间
# 卖出的比例为时间超过 7 天的 ~10%
# 卖出时机
principal = 3000.